In [2]:
import torch
import numpy as np
import pandas as pd
import torch.nn.functional as F
import os
import torch
from kan import KAN
import time
import warnings
warnings.filterwarnings(
    "ignore",
    "CUDA initialization: Unexpected error from cudaGetDeviceCount"
)

##### Loading the inference model and functions

In [3]:
################################setup######################################
def load_csv_data(input_folder: str,
                  train_fname: str,
                  test_fname: str):
    """
    Reads train & test CSVs from disk.
    
    Returns:
      train_df, test_df (both pandas.DataFrame)
    """
    train_path = os.path.join(input_folder, train_fname)
    test_path  = os.path.join(input_folder, test_fname)
    train_df = pd.read_csv(train_path)
    test_df  = pd.read_csv(test_path)
    return train_df, test_df


def extract_features_labels(df: pd.DataFrame):
    """
    Splits a DataFrame into numpy feature array X and label vector y.
    
    The last column is the label.
    """
    X = df.iloc[:, :-1].values
    y = df.iloc[:,  -1].values
    return X, y

# this is a standard PyTorch DataLoader to load the dataset for the training and testing of the model
class DataLoader(object):
    def __init__(self,
                 data,
                 labels,
                 batch_size=1,
                 shuffle=True):
        self.data = data
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = shuffle

    def __len__(self):
        return int(np.ceil(self.data.shape[0] / self.batch_size))

    def __iter__(self):
        n = self.data.shape[0]
        idxlist = list(range(n))
        if self.shuffle:
            np.random.shuffle(idxlist)

        for _, start_idx in enumerate(range(0, n, self.batch_size)):
            end_idx = min(start_idx + self.batch_size, n)
            data = self.data[idxlist[start_idx:end_idx]]
            labels = self.labels[idxlist[start_idx:end_idx]]
            ############################################################
            # Check if any class is missing in the batch
            # present_classes = np.unique(labels.cpu().numpy())
            # all_classes = np.arange(len(label_mapping))  # Adjust based on number of classes
            # missing_classes = set(all_classes) - set(present_classes)
            #
            # if missing_classes:
            #     print(f"Batch {start_idx // self.batch_size} is missing classes {missing_classes}")
            ############################################################
            yield data, labels


class LogitsToPredicate(torch.nn.Module):
    """
    This model has inside a logits model, that is a model which compute logits for the classes given an input example x.
    The idea of this model is to keep logits and probabilities separated. The logits model returns the logits for an example,
    while this model returns the probabilities given the logits model.

    In particular, it takes as input an example x and a class label l. It applies the logits model to x to get the logits.
    Then, it applies a softmax function to get the probabilities per classes. Finally, it returns only the probability related
    to the given class l.
    """

    def __init__(self, logits_model):
        super(LogitsToPredicate, self).__init__()
        self.logits_model = logits_model
        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, x, l, training=False):
        logits = self.logits_model(x, training=training)
        probs = self.softmax(logits)
        out = torch.sum(probs * l, dim=1)  # 计算并返回与给定类标签l对应的概率值
        return out


class MLP(torch.nn.Module):
    """
    This model returns the logits for the classes given an input example. It does not compute the softmax, so the output
    are not normalized.
    This is done to separate the accuracy computation from the satisfaction level computation. Go through the example
    to understand it.
    """

    def __init__(self, layer_sizes):
        super(MLP, self).__init__()
        self.elu = torch.nn.ELU()
        self.dropout = torch.nn.Dropout(0.2)
        self.linear_layers = torch.nn.ModuleList([torch.nn.Linear(layer_sizes[i - 1], layer_sizes[i])
                                                  for i in range(1, len(layer_sizes))])

    def forward(self, x, training=False):
        """
        Method which defines the forward phase of the neural network for our multi class classification task.
        In particular, it returns the logits for the classes given an input example.

        :param x: the features of the example
        :param training: whether the network is in training mode (dropout applied) or validation mode (dropout not applied)
        :return: logits for example x
        """
        for layer in self.linear_layers[:-1]:
            x = self.elu(layer(x))
            if training:
                x = self.dropout(x)
        logits = self.linear_layers[-1](x)
        return logits


class MultiKANModel(torch.nn.Module):
    def __init__(self, kan):
        """
        Wrap an already built MultKAN instance.
        Args:
            kan: a MultKAN model (which has attributes such as act_fun, symbolic_fun, node_bias, node_scale,
                 subnode_bias, subnode_scale, depth, width, mult_homo, mult_arity, input_id, symbolic_enabled, etc.)
        """
        super(MultiKANModel, self).__init__()
        self.kan = kan

    def forward(self, x, training=False, singularity_avoiding=False, y_th=10.):
        # Select input features according to input_id
        x = x[:, self.kan.input_id.long()]
        # Loop through each layer
        for l in range(self.kan.depth):
            # Get outputs from the numerical branch (KANLayer) of current layer
            x_numerical, preacts, postacts_numerical, postspline = self.kan.act_fun[l](x)
            # Get output from the symbolic branch if enabled
            if self.kan.symbolic_enabled:
                x_symbolic, postacts_symbolic = self.kan.symbolic_fun[l](x, singularity_avoiding=singularity_avoiding, y_th=y_th)
            else:
                x_symbolic = 0.
            # Sum the numerical and symbolic outputs
            x = x_numerical + x_symbolic

            # Subnode affine transformation
            x = self.kan.subnode_scale[l][None, :] * x + self.kan.subnode_bias[l][None, :]

            # Process multiplication nodes
            dim_sum = self.kan.width[l+1][0]
            dim_mult = self.kan.width[l+1][1]
            if dim_mult > 0:
                if self.kan.mult_homo:
                    for i in range(self.kan.mult_arity-1):
                        if i == 0:
                            x_mult = x[:, dim_sum::self.kan.mult_arity] * x[:, dim_sum+1::self.kan.mult_arity]
                        else:
                            x_mult = x_mult * x[:, dim_sum+i+1::self.kan.mult_arity]
                else:
                    for j in range(dim_mult):
                        acml_id = dim_sum + int(np.sum(self.kan.mult_arity[l+1][:j]))
                        for i in range(self.kan.mult_arity[l+1][j]-1):
                            if i == 0:
                                x_mult_j = x[:, [acml_id]] * x[:, [acml_id+1]]
                            else:
                                x_mult_j = x_mult_j * x[:, [acml_id+i+1]]
                        if j == 0:
                            x_mult = x_mult_j
                        else:
                            x_mult = torch.cat([x_mult, x_mult_j], dim=1)
                # Concatenate sum and mult parts
                x = torch.cat([x[:, :dim_sum], x_mult], dim=1)

            # Node affine transformation
            x = self.kan.node_scale[l][None, :] * x + self.kan.node_bias[l][None, :]

        # Final x corresponds to the logits output of the whole model
        return x


def save_model(model, model_save_folder, model_name):
    """
    Save the model to disk.
    """
    torch.save(model.state_dict(), os.path.join(model_save_folder, model_name))

    print(f"Model saved to {os.path.join(model_save_folder, model_name)}")


def load_model_state(infer_model, model_save_folder, model_name):
    """
    Load the model from disk.
    """
    checkpoint = torch.load(
        os.path.join(model_save_folder, model_name),
        map_location=device,
        weights_only=True     # <-- only load tensor weights, no pickle objects
    )
    infer_model.load_state_dict(checkpoint)
    infer_model.eval()
    return infer_model


def compute_accuracy(loader, model):
    total_correct = 0
    total_samples = 0
    for data, labels in loader:
        logits = model(data)
        preds = torch.argmax(logits, dim=1)
        total_correct += (preds == labels).sum()
        total_samples += labels.numel()
    return total_correct.float() / total_samples


def compute_sat_levels(loader, P):
	sat_level  = 0
	for data, labels in loader:
		x = ltn.Variable("x", data)
		x_MQTT_DDoS_Connect_Flood = ltn.Variable("x_MQTT_DDoS_Connect_Flood", data[labels == 0])
		x_MQTT_DDoS_Publish_Flood = ltn.Variable("x_MQTT_DDoS_Publish_Flood", data[labels == 1])
		x_MQTT_DoS_Connect_Flood = ltn.Variable("x_MQTT_DoS_Connect_Flood", data[labels == 2])
		x_MQTT_DoS_Publish_Flood = ltn.Variable("x_MQTT_DoS_Publish_Flood", data[labels == 3])
		x_MQTT_Malformed_Data = ltn.Variable("x_MQTT_Malformed_Data", data[labels == 4])
		x_Benign = ltn.Variable("x_Benign", data[labels == 5])

		sat_level = SatAgg(
			Forall(x_MQTT_DDoS_Connect_Flood, P(x_MQTT_DDoS_Connect_Flood, l_MQTT_DDoS_Connect_Flood)),
			Forall(x_MQTT_DDoS_Publish_Flood, P(x_MQTT_DDoS_Publish_Flood, l_MQTT_DDoS_Publish_Flood)),
			Forall(x_MQTT_DoS_Connect_Flood, P(x_MQTT_DoS_Connect_Flood, l_MQTT_DoS_Connect_Flood)),
			Forall(x_MQTT_DoS_Publish_Flood, P(x_MQTT_DoS_Publish_Flood, l_MQTT_DoS_Publish_Flood)),
			Forall(x_MQTT_Malformed_Data, P(x_MQTT_Malformed_Data, l_MQTT_Malformed_Data)),
			Forall(x_Benign, P(x_Benign, l_Benign))
		)
	return sat_level


##############################Load data######################################
# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")  # Use CPU for this example
print(f"\n Using device: {device} \n")

# Load data
input_folder = '/home/zyang44/Github/baseline_cicIOT/P1_structurelevel/efficiency/input_files'
train_fname = 'logiKNet_train_35945.csv'
test_fname = 'logiKNet_test_3994.csv'

train_df, test_df = load_csv_data(input_folder, train_fname, test_fname)
# Extract features and labels   
X_train, y_train = extract_features_labels(train_df)
X_test, y_test = extract_features_labels(test_df)

dataset_numeric = {
    'train_input': torch.tensor(X_train, dtype=torch.float32, device=device),
    'train_label': torch.tensor(y_train, dtype=torch.long, device=device),
    'test_input': torch.tensor(X_test, dtype=torch.float32, device=device),
    'test_label': torch.tensor(y_test, dtype=torch.long, device=device)
}

train_loader = DataLoader(
    dataset_numeric['train_input'],
    dataset_numeric['train_label'], 
    batch_size=len(X_train), 
    shuffle=True
    )
test_loader = DataLoader(
    dataset_numeric['test_input'],
    dataset_numeric['test_label'],
    # batch_size=32,
    shuffle=False
    )


###############################load model and testing########################################
model_state_folder = '/home/zyang44/Github/baseline_cicIOT/P1_structurelevel/efficiency/model_weights'

# load all four models
mlp_infer = MLP(layer_sizes=(18, 10, 6)).to(device)
mlp_infer = load_model_state(mlp_infer, model_state_folder, 'mlp.pt')

logicmlp_infer = MLP(layer_sizes=(18, 10, 6)).to(device)
logicmlp_infer = load_model_state(logicmlp_infer, model_state_folder, 'logic_mlp.pt')

logiKNet_infer = KAN(width=[18, 10, 6], grid=5, k=3, seed=42, device=device)
logiKNet_infer = load_model_state(logiKNet_infer, model_state_folder, 'logiKNet.pt')

hierarchical_logiKNet_infer = KAN(width=[18, 10, 6], grid=5, k=3, seed=42, device=device)
hierarchical_logiKNet_infer = load_model_state(hierarchical_logiKNet_infer, model_state_folder, 'hierarchical_logiKNet.pt')

model_list = {
    'mlp': mlp_infer,
    'logic_mlp': logicmlp_infer,
    'logiKNet': logiKNet_infer,
    'hierarchical_logiKNet': hierarchical_logiKNet_infer
}

# test the models 
def test_model(model, loader, model_name=""):
    start_time = time.perf_counter()

    model.eval()
    batch_times = []
    with torch.no_grad():
        for data, labels in loader:
            batch_start = time.perf_counter()
            logits = model(data)
            preds = torch.argmax(logits, dim=1)
            batch_end = time.perf_counter()
            batch_times.append(batch_end - batch_start)

    if batch_times:
        mean_time = sum(batch_times) / len(batch_times)
        print(f"[{model_name}] Mean batch inference time: {mean_time:.4f} seconds")
    else:
        print(f"[{model_name}] No batches to measure.") 

    end_time = time.perf_counter()
    print(f"[{model_name}] Inference time: {end_time - start_time:.4f} seconds")


# for model_name, model in model_list.items():
#     test_model(model, test_loader, model_name)


 Using device: cpu 

checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0


#### Persistent Processing Power vs Rate

##### Statistics for mean inference time

In [4]:
jetson_stat = {
    'Inference': [0.68151165, 0.67085658, 159.49545998, 158.91916464],
    'Power': [1460.3, 1340.9, 1034.2, 1042.2],
    'Background Power': 4850
}

- **For mlp/logic_mlp**: set unit time to 1 ms.
- **For logiKNet/h-logiKNet**: set unit time to 200 ms.

- Persistent: detection software always running, the mean power consumption is: 
  **jetson_stat['Backgournd Power'] + jetson_stat['Power']**
- Intermittent: dectect software run every 5 unit of flow time, the mean power consumption is:
  **(5 * jetson_stat['Backgournd Power'] + jetson_stat['Inference'] * jetson_stat['Power']) / 5**

#### Calculate Mean Power Comsumption

In [5]:
# Calculate the mean power consumption for persistent and intermittent modes
model_names = ['mlp', 'logic_mlp', 'logiKNet', 'hierarchical_logiKNet']

# Initialize dictionary to store power consumption results
power_consumption = {
    'persistent': {},
    'intermittent': {}
}

n = 5  # Intermittent mode runs every n units of flow time

# Calculate power consumption for each model
for i, model_name in enumerate(model_names):
    # Persistent mode: detection software always running
    # Mean power = Background Power + Power
    persistent_power = jetson_stat['Background Power'] + jetson_stat['Power'][i]
    
    # Intermittent mode: detection software runs every n units of flow time
    # Mean power = (n * Background Power + Inference * Power) / n
    if model_name == 'mlp' or model_name == 'logic_mlp':
        intermittent_power = (n * jetson_stat['Background Power'] +
                             jetson_stat['Inference'][i] * jetson_stat['Power'][i]) / n
    else:
        # For logiKNet and hierarchical_logiKNet, use the same formula
        # Mean power = (200n * Background Power + Inference * Power) / 200n
        intermittent_power = (200 * n * jetson_stat['Background Power'] +
                             jetson_stat['Inference'][i] * jetson_stat['Power'][i]) / (200 * n)

    # Store results
    power_consumption['persistent'][model_name] = persistent_power
    power_consumption['intermittent'][model_name] = intermittent_power

# Display results
print("Power Consumption Analysis (mW)")
print("=" * 60)
print(f"{'Model':<20} {'Persistent':<12} {'Intermittent':<12} {'Savings':<10}")
print("-" * 60)

for model_name in model_names:
    persistent = power_consumption['persistent'][model_name]
    intermittent = power_consumption['intermittent'][model_name]
    savings = ((persistent - intermittent) / persistent) * 100
    
    print(f"{model_name:<20} {persistent:<12.1f} {intermittent:<12.1f} {savings:<10.1f}%")

print("\nPower Consumption Dictionary:")
print(power_consumption)

Power Consumption Analysis (mW)
Model                Persistent   Intermittent Savings   
------------------------------------------------------------
mlp                  6310.3       5049.0       20.0      %
logic_mlp            6190.9       5029.9       18.8      %
logiKNet             5884.2       5015.0       14.8      %
hierarchical_logiKNet 5892.2       5015.6       14.9      %

Power Consumption Dictionary:
{'persistent': {'mlp': 6310.3, 'logic_mlp': 6190.9, 'logiKNet': 5884.2, 'hierarchical_logiKNet': 5892.2}, 'intermittent': {'mlp': 5049.042292499, 'logic_mlp': 5029.910317624401, 'logiKNet': 5014.9502047113165, 'hierarchical_logiKNet': 5015.6255533878075}}


#### Calculate Rate

- The unit time must longer than model single inference duration. Because we didn't have multi-task running power consumption.
- The threat rate is to see how many malicious case missed, two options here: **false negative Benign is counted** or not.

{"MQTT-DDoS-Connect_Flood": 0, 
"MQTT-DDoS-Publish_Flood": 1, 
"MQTT-DoS-Connect_Flood": 2, 
"MQTT-DoS-Publish_Flood": 3, 
"MQTT-Malformed_Data": 4, 
"Benign": 5} 

In [7]:
# ============================================================================
# COMPREHENSIVE THREAT RATE ANALYSIS WITH INTERMITTENT INFERENCE
# ============================================================================

def calculate_intermittent_threat_rate(model, test_loader, intermit_interval=2):
    """
    Calculate the threat rate of a model with intermittent inference.
    Threat rate is calculated based on ALL test cases, but only sampled cases get predictions.
    All non-sampled cases are treated as missed (false negatives).
    """
    model.eval()
    
    # Get all test data
    all_data = []
    all_labels = []
    
    for data, labels in test_loader:
        all_data.append(data)
        all_labels.append(labels)
    
    # Concatenate all batches
    all_data = torch.cat(all_data, dim=0)
    all_labels = torch.cat(all_labels, dim=0)
    
    total_cases = len(all_data)
    
    # Initialize predictions array - all start as "missed" (predict as Benign=5)
    all_preds = np.full(total_cases, 5)  # Default to Benign class
    sampled_indices = []
    
    # Handle persistent mode (intermit_interval=1)
    if intermit_interval == 1:
        # Process all cases (persistent mode)
        with torch.no_grad():
            logits = model(all_data)
            preds = torch.argmax(logits, dim=1)
            all_preds = preds.cpu().numpy()
            sampled_indices = list(range(total_cases))
    else:
        # Intermittent mode - Do inference on case 0 as initialization
        if total_cases > 0:
            with torch.no_grad():
                logits = model(all_data[0:1])
                pred = torch.argmax(logits, dim=1)
                all_preds[0] = pred.cpu().numpy()[0]
                sampled_indices.append(0)
        
        # Process remaining cases in groups
        current_idx = 1
        while current_idx < total_cases:
            group_end = min(current_idx + intermit_interval, total_cases)
            group_indices = list(range(current_idx, group_end))
            
            if len(group_indices) > 0:
                selected_idx = np.random.choice(group_indices)
                sampled_indices.append(selected_idx)
                
                with torch.no_grad():
                    logits = model(all_data[selected_idx:selected_idx+1])
                    pred = torch.argmax(logits, dim=1)
                    all_preds[selected_idx] = pred.cpu().numpy()[0]
            
            current_idx = group_end
    
    # Convert to numpy arrays
    all_preds = np.array(all_preds)
    all_labels = all_labels.cpu().numpy()
    
    # Calculate threat rate on ALL cases
    malicious_mask = all_labels != 5
    malicious_labels = all_labels[malicious_mask]
    malicious_preds = all_preds[malicious_mask]
    
    missed_malicious = np.sum(malicious_labels != malicious_preds)
    total_malicious = len(malicious_labels)
    
    if total_malicious == 0:
        return 0.0, {}
    
    threat_rate = missed_malicious / total_malicious
    
    # Calculate sampling statistics
    sampling_stats = {
        'total_cases': total_cases,
        'sampled_cases': len(sampled_indices),
        'sampling_rate': len(sampled_indices) / total_cases,
        'total_malicious': total_malicious,
        'missed_malicious': missed_malicious
    }
    
    return threat_rate, sampling_stats


def run_comprehensive_threat_analysis(model_list, test_loader, intermit_intervals=[1, 2, 3, 5]):
    """Run comprehensive threat rate analysis for different intermit intervals."""
    results = {}
    
    print("Comprehensive Threat Rate Analysis")
    print("=" * 80)
    
    for interval in intermit_intervals:
        print(f"\nAnalyzing Intermit Interval: {interval}")
        print("-" * 50)
        
        results[interval] = {}
        
        for model_name, model in model_list.items():
            # Set random seed for reproducibility
            np.random.seed(42)
            
            threat_rate, stats = calculate_intermittent_threat_rate(
                model, test_loader, intermit_interval=interval
            )
            
            results[interval][model_name] = {
                'threat_rate': threat_rate,
                'sampling_stats': stats
            }
            
            mode_str = "Persistent" if interval == 1 else f"Intermittent (interval={interval})"
            print(f"{model_name:<20} | {mode_str:<25} | Threat Rate: {threat_rate:.4f} ({threat_rate*100:.2f}%)")
            
            if interval > 1:
                print(f"{'':>20} | {'':>25} | Sampled: {stats['sampled_cases']}/{stats['total_cases']} ({stats['sampling_rate']*100:.1f}%)")
    
    return results


def display_comparison_table(results, intermit_intervals=[1, 2, 3, 5]):
    """Display a comprehensive comparison table of threat rates."""
    print("\n" + "=" * 100)
    print("THREAT RATE COMPARISON TABLE")
    print("=" * 100)
    
    # Header
    header = f"{'Model':<20}"
    for interval in intermit_intervals:
        mode_str = "Persistent" if interval == 1 else f"Interval={interval}"
        header += f" | {mode_str:<12}"
    header += " | Max Increase"
    print(header)
    print("-" * 100)
    
    # Data rows
    for model_name in results[intermit_intervals[0]].keys():
        row = f"{model_name:<20}"
        threat_rates = []
        
        for interval in intermit_intervals:
            threat_rate = results[interval][model_name]['threat_rate']
            threat_rates.append(threat_rate)
            row += f" | {threat_rate:.4f}"
        
        # Calculate maximum increase from persistent mode
        persistent_rate = threat_rates[0]  # interval=1 is persistent
        max_increase = max([(rate - persistent_rate) / persistent_rate * 100 
                          for rate in threat_rates[1:]] if len(threat_rates) > 1 else [0])
        
        row += f" | {max_increase:>10.1f}%"
        print(row)


def save_results_to_file(results, filename='threat_rate_analysis.txt'):
    """Save the comprehensive results to a text file."""
    with open(filename, 'w') as f:
        f.write("Comprehensive Threat Rate Analysis Results\n")
        f.write("=" * 80 + "\n\n")
        
        for interval in sorted(results.keys()):
            f.write(f"Intermit Interval: {interval}\n")
            f.write("-" * 30 + "\n")
            
            for model_name, data in results[interval].items():
                threat_rate = data['threat_rate']
                stats = data['sampling_stats']
                
                f.write(f"Model: {model_name}\n")
                f.write(f"  Threat Rate: {threat_rate:.6f} ({threat_rate*100:.2f}%)\n")
                f.write(f"  Total Cases: {stats['total_cases']}\n")
                f.write(f"  Sampled Cases: {stats['sampled_cases']}\n")
                f.write(f"  Sampling Rate: {stats['sampling_rate']:.4f} ({stats['sampling_rate']*100:.1f}%)\n")
                f.write(f"  Total Malicious: {stats['total_malicious']}\n")
                f.write(f"  Missed Malicious: {stats['missed_malicious']}\n")
                f.write("\n")
            f.write("\n")
    
    print(f"Results saved to: {filename}")


# ============================================================================
# MAIN EXECUTION
# ============================================================================

# Check if this is the first run to avoid duplicate execution
import sys
if not hasattr(sys, '_threat_analysis_executed'):
    print("Starting Comprehensive Threat Rate Analysis...")
    print("Testing intermit intervals: [1, 2, 3, 5]")
    print("- Interval 1 = Persistent mode (all cases processed)")
    print("- Intervals 2,3,5 = Intermittent mode (sampled cases only)")
    print("\n")

    # Run the comprehensive analysis
    intermit_intervals = [1, 2, 3, 5]
    results = run_comprehensive_threat_analysis(model_list, test_loader, intermit_intervals)

    # Display comparison table
    display_comparison_table(results, intermit_intervals)

    # Save results to file
    save_results_to_file(results, 'threat_rate_analysis.txt')

    print(f"\nFinal Results Dictionary:")
    print("=" * 50)
    for interval in intermit_intervals:
        print(f"Interval {interval}:")
        for model_name in results[interval]:
            threat_rate = results[interval][model_name]['threat_rate']
            print(f"  {model_name}: {threat_rate:.6f}")
        print()
    
    # Mark as executed to prevent duplicate runs
    sys._threat_analysis_executed = True
    
else:
    print("Analysis already executed. Results are available in the 'results' variable.")
    print("To run again, restart the kernel or delete the execution flag with:")
    print("del sys._threat_analysis_executed")

Starting Comprehensive Threat Rate Analysis...
Testing intermit intervals: [1, 2, 3, 5]
- Interval 1 = Persistent mode (all cases processed)
- Intervals 2,3,5 = Intermittent mode (sampled cases only)


Comprehensive Threat Rate Analysis

Analyzing Intermit Interval: 1
--------------------------------------------------
mlp                  | Persistent                | Threat Rate: 0.4421 (44.21%)


logic_mlp            | Persistent                | Threat Rate: 0.3871 (38.71%)
logiKNet             | Persistent                | Threat Rate: 0.2440 (24.40%)
hierarchical_logiKNet | Persistent                | Threat Rate: 0.2349 (23.49%)

Analyzing Intermit Interval: 2
--------------------------------------------------
mlp                  | Intermittent (interval=2) | Threat Rate: 0.7226 (72.26%)
                     |                           | Sampled: 1998/3994 (50.0%)
logic_mlp            | Intermittent (interval=2) | Threat Rate: 0.6937 (69.37%)
                     |                           | Sampled: 1998/3994 (50.0%)
logiKNet             | Intermittent (interval=2) | Threat Rate: 0.6244 (62.44%)
                     |                           | Sampled: 1998/3994 (50.0%)
hierarchical_logiKNet | Intermittent (interval=2) | Threat Rate: 0.6205 (62.05%)
                     |                           | Sampled: 1998/3994 (50.0%)

Analyzing Intermit Interval: 3
-----------